# Task 6 : 从数据集中提取样本，检查数据类型

In [3]:
import os,json,sys,logging
sys.path.append("./share")
sys.path.append("./common")
import pandas as pd
import json
from tqdm import tqdm
from IoTCommon import CIoTCommon
from IoTTotalFeature import CIoTTotalFeature
from IoTSample import CIoTSample
from Config import g_data_root
import warnings
warnings.simplefilter("ignore")
g_sample_root = "%ssample"%g_data_root

### 检测数据集类型

In [4]:
df_features = pd.read_excel("%s/features/features.xlsx"%g_data_root,index_col=0).reset_index()
df_features = df_features[df_features['effective'] !="N"]
df_features = df_features[df_features['count'] > 1].reset_index(drop=True)
totalFeature = CIoTTotalFeature()
df_features = CIoTTotalFeature.get_expand_features(df_features,totalFeature)

In [14]:
def check_columns(trafficType,protocol,feature,type):
    mask=df_features['feature']==feature
    df_tmp = df_features[mask]
    tmp = {}
    tmp['trafficType'] = trafficType
    tmp['protocol'] = protocol
    tmp['feature'] = feature
    if df_tmp.shape[0] <= 0:
        tmp['result'] = "Not Found"
    else:
        tmp['configure_type']=df_tmp.iloc[0]['type']
        if tmp['configure_type'] in ['int']:
            tmp['configure_type'] = "int64"
            
        if tmp['configure_type'] in ['binary','string']:
            tmp['configure_type'] = "object"
            
        tmp['current_type'] = type
        tmp['values']=df_tmp.iloc[0]['values']
        tmp['result'] = "succeed" if tmp['current_type'] == tmp['configure_type'] else "failed" 
    
    return tmp
    
def get_check_result(trafficType1 = None, protocol1 = None, feature1 = None):
    all_data = []
    test = CIoTSample()
    for trafficType in test.get_attack_type():
        if trafficType1 and trafficType !=trafficType1 :continue
        for protocol in tqdm(test.get_attack_protocol(trafficType),desc="loading attack samples"):
            if protocol1 and protocol != protocol1 :continue
            df_test = test.get_attack_sample(trafficType,protocol)
            df_test = test.fill_na(df_test)
            for col in df_test.keys():
                if feature1 and col != feature1:continue
                type = df_test.dtypes[col]
                tmp = check_columns(trafficType,protocol,col,type)
                all_data.append(tmp)
        
    for trafficType in test.get_sensor_type():
        if trafficType1 and trafficType !=trafficType1 :continue
        for protocol in tqdm(test.get_sensor_protocol(trafficType),desc="loading normal samples"):
            if protocol1 and protocol != protocol1 :continue
            df_test = test.get_sensor_sample(trafficType,protocol)
            df_test = test.fill_na(df_test)
            for col in df_test.keys():
                if feature1 and col != feature1:continue
                type = df_test.dtypes[col]
                tmp = check_columns(trafficType,protocol,col,type)
                all_data.append(tmp)
            
    df_result = pd.DataFrame(all_data)
    return df_result

In [15]:
trafficType="Backdoor_attack"
protocol="eth:ethertype:ip:udp:dns"
feature = 'dns.retransmission'
df_result = get_check_result(trafficType,protocol,feature)

loading attack samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 273.36it/s]


In [16]:
df_result

,trafficType,protocol,feature,configure_type,current_type,values,result
0,Backdoor_attack,eth:ethertype:ip:udp:dns,dns.retransmission,bool,bool,"[nan, '1']",succeed


In [17]:
df_result = get_check_result()

loading normal samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:21<00:00,  7.08s/it]


In [18]:
df_result.to_csv("./test.csv")

In [19]:
df_result

,trafficType,protocol,feature,result,configure_type,current_type,values
0,Port Scanning attack,eth:ethertype:ip:tcp,id,Not Found,NaN,NaN,NaN
1,Port Scanning attack,eth:ethertype:ip:tcp,Label,Not Found,NaN,NaN,NaN
2,Port Scanning attack,eth:ethertype:ip:tcp,protocol,Not Found,NaN,NaN,NaN
3,Port Scanning attack,eth:ethertype:ip:tcp,frame.time_utc,failed,datetime,int64,"['Nov 24, 2021 15:03:50.536853000 UTC', 'Nov 2..."
4,Port Scanning attack,eth:ethertype:ip:tcp,frame.time_delta,succeed,int64,int64,"['0.036697000', '0.003410000', '0.163624000', ..."
...,...,...,...,...,...,...,...
4750,Temperature_and_Humidity,eth:ethertype:ip:tcp:tls,tls.handshake.ja4_r,failed,category,int64,"['t13d1516h2_002f,0035,009c,009d,1301,1302,130..."
4751,Temperature_and_Humidity,eth:ethertype:ip:tcp:tls,tls.handshake.extensions_ec_point_format,succeed,bool,bool,"['0', nan]"
4752,Temperature_and_Humidity,eth:ethertype:ip:tcp:tls,tls.handshake.ciphersuite,failed,category,int64,"['0x0035', nan, '0x00ff', '0x009c']"
4753,Temperature_and_Humidity,eth:ethertype:ip:tcp:tls,tls.handshake.extensions_server_name,failed,category,int64,"['accounts.google.com', nan, 'update.googleapi..."
